In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import yfinance as yf

# Advanced Pandas

Now that you have gotten familiar with using pandas, we will cover some more advanced concepts with pandas. 

## Missing Data

Often, there is missing data in your dataset and you need to know how to be able to handle these values. First, we'll go through how to find missing data.

In [2]:
# Employee data on hours worked today
d = {'Name': ['Adam', 'Sally', 'Carter', 'George', 'Fiona'],
     'Age': [18, 19, 17, np.nan, 18],
     'Sex': ['M', 'F', np.nan, 'M', 'F'],
     'Hours': [8, 7.5, 4, 7, np.nan]}

data = pd.DataFrame(d)

data.head()

,Name,Age,Sex,Hours
0,Adam,18.0,M,8.0
1,Sally,19.0,F,7.5
2,Carter,17.0,NaN,4.0
3,George,NaN,M,7.0
4,Fiona,18.0,F,NaN


We can see that we have some missing values since this is a small dataset but if it was a larger dataset and you wanted to extract all missing values, a tool would be helpful.

In [3]:
data.isnull()

,Name,Age,Sex,Hours
0,False,False,False,False
1,False,False,False,False
2,False,False,True,False
3,False,True,False,False
4,False,False,False,True


In [4]:
data.notnull()

,Name,Age,Sex,Hours
0,True,True,True,True
1,True,True,True,True
2,True,True,False,True
3,True,False,True,True
4,True,True,True,False


This first tells us which values are null. The second shows the values that are not null. 

In [5]:
# When is Age NULL
data[pd.isnull(data['Age'])]

,Name,Age,Sex,Hours
3,George,NaN,M,7.0


In [6]:
# When is Sex NULL
data[pd.isnull(data['Sex'])]

,Name,Age,Sex,Hours
2,Carter,17.0,NaN,4.0


In [7]:
# When is Hours Worked NULL
data[pd.isnull(data['Hours'])]

,Name,Age,Sex,Hours
4,Fiona,18.0,F,NaN


Now that you have found the missing values, the next step is to determine how to handle these values. You need to investigate why this missing value exists. Was there an issue in the data collection process? Did someone choose not to answer a voluntary question on a survey? Was data lost? Does the data not exist? 

Some of the ways to handle the values are:
* Delete the missing value: Ideally you don't want to do this because you will lose data that could be important. If you have a large dataset and this data makes up less than 5% of the data, then it may be okay, but you want to make sure you aren't losing important data from the data that is available for these records.

- Replace with arbitrary value
- Replace with mean
- Replace with median
- Forward fill (method = 'ffill')
- Backward fill (method = 'bfill')

You can read more on handling missing values [here](https://www.analyticsvidhya.com/blog/2021/10/handling-missing-value/).

### Dropping missing values

In [8]:
data.dropna()

,Name,Age,Sex,Hours
0,Adam,18.0,M,8.0
1,Sally,19.0,F,7.5


This does not update our data since we left `inplace=False` as the default. If we were to drop the missing values, this is what our data would look like. It drops the missing values that exist in ANY column.

In [9]:
data.dropna(how='all')

,Name,Age,Sex,Hours
0,Adam,18.0,M,8.0
1,Sally,19.0,F,7.5
2,Carter,17.0,NaN,4.0
3,George,NaN,M,7.0
4,Fiona,18.0,F,NaN


This would drop rows where all the data is missing. 

In [10]:
data.dropna(axis=0, how='any')

,Name,Age,Sex,Hours
0,Adam,18.0,M,8.0
1,Sally,19.0,F,7.5


This drops the row where there is at least 1 null value in any column. Similarly, you can do this for any columns by setting `axis=1`.

### Replace with an arbitrary value

When hours worked is NULL, that means the employee did not work that day. Hence, we want to replace the missing values with a 0. 

In [11]:
data.Hours.fillna(0, inplace=True)

In [12]:
data

,Name,Age,Sex,Hours
0,Adam,18.0,M,8.0
1,Sally,19.0,F,7.5
2,Carter,17.0,NaN,4.0
3,George,NaN,M,7.0
4,Fiona,18.0,F,0.0


Similarly, when no sex is given, we want to replace this missing value with 'No Sex'

In [13]:
data.Sex.fillna('No Sex', inplace=True)

data

,Name,Age,Sex,Hours
0,Adam,18.0,M,8.0
1,Sally,19.0,F,7.5
2,Carter,17.0,No Sex,4.0
3,George,NaN,M,7.0
4,Fiona,18.0,F,0.0


### Replace with median

Lastly, we have a missing value in the Age. We'll go through a few methods for how we can deal with age. For when we are missing the age, we can fill it with the median so that it is more robust to outliers. 

In [14]:
data.Age

0    18.0
1    19.0
2    17.0
3     NaN
4    18.0
Name: Age, dtype: float64

In [15]:
data.Age.fillna(data.Age.mean())

0    18.0
1    19.0
2    17.0
3    18.0
4    18.0
Name: Age, dtype: float64

### Forward and Backwards Fill

Let's assume we want to use forward or backward fill for replacing the Age data. This wouldn't be an appropriate method to use in general since ages can vary and the next/last value may not be indicative of the missing value. We are simply using it here to show an example of the two methods. 

Forward Fill: Uses the last valid observation to fill the NA. 

Backward Fill: Uses the next valid observation to fill the NA. 

In [16]:
data.head()

,Name,Age,Sex,Hours
0,Adam,18.0,M,8.0
1,Sally,19.0,F,7.5
2,Carter,17.0,No Sex,4.0
3,George,NaN,M,7.0
4,Fiona,18.0,F,0.0


In [17]:
# Forward Fill
data.fillna(method='ffill')

,Name,Age,Sex,Hours
0,Adam,18.0,M,8.0
1,Sally,19.0,F,7.5
2,Carter,17.0,No Sex,4.0
3,George,17.0,M,7.0
4,Fiona,18.0,F,0.0


Notice how the missing age is George now has 17 as the Age since that was the value before George. 

In [18]:
# Backward Fill
data.fillna(method='bfill')

,Name,Age,Sex,Hours
0,Adam,18.0,M,8.0
1,Sally,19.0,F,7.5
2,Carter,17.0,No Sex,4.0
3,George,18.0,M,7.0
4,Fiona,18.0,F,0.0


Similarly, notice how the missing age is George now has 18 as the Age since that was the value after George. 

___

## Reading in Data

We have learned how to read in data from dictionaries and Series. You can also read in data as a DataFrame, for example from a csv file.

In [19]:
# read in daily activity csv
activity = pd.read_csv('daily_activity.csv', index_col='Date')

In [20]:
# View our data
activity.head()

,Walk,Swim,Running
Date,,,
2021-01-01,89,36,26
2021-01-02,78,39,29
2021-01-03,68,39,27
2021-01-04,93,30,26
2021-01-05,68,26,26


In [21]:
# Columns
activity.columns

Index(['Walk', 'Swim', 'Running'], dtype='object')

In [22]:
# Reset the index 
activity.reset_index(inplace=True)

In [23]:
activity.head()

,Date,Walk,Swim,Running
0,2021-01-01,89,36,26
1,2021-01-02,78,39,29
2,2021-01-03,68,39,27
3,2021-01-04,93,30,26
4,2021-01-05,68,26,26


As you can see, it creates a new column in the DataFrame for what used to be the index.

In [24]:
# Set the index to be date
activity.set_index('Date', inplace=True)

The `inplace=True` parameter updates your current DataFrame. Without that parameter, the default is False. 

In [25]:
# View only land activity
activity[['Walk', 'Running']]

,Walk,Running
Date,,
2021-01-01,89,26
2021-01-02,78,29
2021-01-03,68,27
2021-01-04,93,26
2021-01-05,68,26
...,...,...
2021-04-26,68,26
2021-04-27,78,27
2021-04-28,78,27


Now, let's say that we are a workout enthusiast and we also have the time to work out in the gym but that isn't stored in this dataframe. We read in another dataframe called `gym.csv` that stores the time we spend in the gym.

In [26]:
gym = pd.read_csv('gym.csv')

In [27]:
# View our data
gym.head()

,Unnamed: 0,Date,Time
0,0,2021-01-01,27
1,1,2021-01-02,30
2,2,2021-01-03,27
3,3,2021-01-04,25
4,4,2021-01-05,26


In [28]:
# Select columns
gym = gym[['Date', 'Time']]

## Concat

Now we want all of our data in one table, so we need to join the two tables together. To do this, we need to have the same indices so that we can do an inner join on the index, which basically means that we are matching our data on the dates that both tables have. Hence, if any date does not appear in one of the tables, it would not show up in our inner join. 

To do this, we need to set our index to Date so that it matches the activity table. Then we use `pd.concat([df1, df2])` to join where we specify the parameter `join=inner` to show it is in an inner join and `axis=1` to show we are adding another column into our new table.

In [29]:
# Set the index
gym.set_index('Date', inplace=True)

In [30]:
# Join the tables
daily_activity = pd.concat([activity, gym], join='inner', axis=1)

daily_activity.head()

,Walk,Swim,Running,Time
Date,,,,
2021-01-01,89,36,26,27
2021-01-02,78,39,29,30
2021-01-03,68,39,27,27
2021-01-04,93,30,26,25
2021-01-05,68,26,26,26


In [31]:
len(daily_activity), len(activity), len(gym)

(120, 120, 120)

We can see that we have a new column called Time from our Gym table and that we had all the same dates between the two tables so they matched up. Next, `Time` makes sense as a column when it is just a gym table, but with this new table, we want to rename our columns so that it is `Gym` instead. 

In [32]:
# Rename column
daily_activity.columns = ['Walk', 'Swim', 'Run', 'Gym']

You simply pass in a list of names to rename the columns by setting `df.columns`. In this case, your list of column names needs to match the number of columns in your table.

Now, lastly, I want to create a new DataFrame that summarizes the average number of minutes I walked, swam, ran and went to the gym for this time period. We will do this by creating a dictionary with our new values.

In [33]:
activity_dic = {'Walk': np.mean(daily_activity.Walk),
                'Run': np.mean(daily_activity.Run),
                'Swim': np.mean(daily_activity.Swim),
                'Gym': np.mean(daily_activity.Gym)
               }

In [34]:
activity_stats= pd.DataFrame(activity_dic, index=[0])

Now what if you wanted to have the activity as a row and the mean as the column? You would transpose your matrix in order to do that.

In [35]:
activity_stats = activity_stats.T

activity_stats

,0
Walk,76.041667
Run,27.458333
Swim,32.391667
Gym,25.333333


In [36]:
# Rename column
activity_stats.columns = ['mean']

This was mainly to show you that sometimes you need to transpose your data when creating a DataFrame from a dictionary and that basically means swapping your rows and columns. 

A faster way to get the means for each activity type would be to do the following:

In [37]:
pd.DataFrame(daily_activity.describe().loc['mean'])

,mean
Walk,76.041667
Swim,32.391667
Run,27.458333
Gym,25.333333


___

## Data Types

Lastly, we want to look into how to convert datatypes, specifically dates.

In [38]:
daily_activity.reset_index().dtypes

Date    object
Walk     int64
Swim     int64
Run      int64
Gym      int64
dtype: object

You'll see that with our data from the csv, we have dates as an object. What if we wanted to convert our dates to datetime? datetime is a data type that has a lot of easy extraction functions for dates, i.e. extrating the year, month, day, etc.

This can be done very easily with DataFrames!

In [39]:
daily_activity.index = pd.to_datetime(daily_activity.index)

In [40]:
daily_activity.reset_index().dtypes

Date    datetime64[ns]
Walk             int64
Swim             int64
Run              int64
Gym              int64
dtype: object

All done! Let's take a look at some of the functionality with `datetime` objects.

In [41]:
daily_activity.head()

,Walk,Swim,Run,Gym
Date,,,,
2021-01-01,89,36,26,27
2021-01-02,78,39,29,30
2021-01-03,68,39,27,27
2021-01-04,93,30,26,25
2021-01-05,68,26,26,26


In [42]:
daily_activity.index[0]

Timestamp('2021-01-01 00:00:00')

In [43]:
daily_activity.index[0].month

1

Remember to check data types when you are trying to join data together using either concat or inner joins. For example, when we joined our daily activity and gym data together, we were able to since both Date indices were an `object`; however, if one was an `object` while the other was a `datetime`, we would have to convert it.

For graphing purposes, datetime is more ideal.

___

## Grouping 

We can use `group_by` functions to organize your data and get summary statistics by grouping single or multiple categories together.

In [44]:
daily_activity.groupby(['Walk']).mean().head()

,Swim,Run,Gym
Walk,,,
55,32.000000,28.666667,21.666667
57,32.666667,29.333333,27.000000
58,32.333333,27.333333,26.333333
59,25.000000,28.000000,25.000000
60,33.000000,28.500000,29.500000


Notice how our index is no longer time. It is now grouped together by the Walk values, i.e. 55 minutes, 57 minutes, etc. Then based on those values, we have taken the mean time spent Swimming, Running or at the Gym. 

Reading this table would go anytime you walked 55 minutes, on average you also swam 32 minutes, ran 29 minutes and went to the gym for 22 minutes. With some other examples, we can go through group by as well to see its functionality in other applications.

___

## Sorting

Sorting is helpful if you want to organize your values by ascending or descending a certain columns, or a list of columns.

In [45]:
daily_activity.sort_values('Walk')

,Walk,Swim,Run,Gym
Date,,,,
2021-02-23,55,33,28,23
2021-02-04,55,28,29,22
2021-01-14,55,35,29,20
2021-03-17,57,27,30,29
2021-01-08,57,37,29,27
...,...,...,...,...
2021-02-28,94,29,26,21
2021-04-24,94,27,25,25
2021-02-08,94,38,29,26


Our values are sorted based on Walk in ascending order. We can also sort in descending order and with multiple columns. 

In [46]:
daily_activity.sort_values('Walk', ascending=False)

,Walk,Swim,Run,Gym
Date,,,,
2021-01-26,95,39,25,26
2021-04-16,95,28,30,28
2021-04-24,94,27,25,25
2021-02-08,94,38,29,26
2021-04-07,94,36,25,20
...,...,...,...,...
2021-03-17,57,27,30,29
2021-01-12,57,34,29,25
2021-02-23,55,33,28,23


Now, we have descending order.

In [47]:
daily_activity.sort_values(['Walk', 'Swim'])

,Walk,Swim,Run,Gym
Date,,,,
2021-02-04,55,28,29,22
2021-02-23,55,33,28,23
2021-01-14,55,35,29,20
2021-03-17,57,27,30,29
2021-01-12,57,34,29,25
...,...,...,...,...
2021-02-28,94,29,26,21
2021-04-07,94,36,25,20
2021-02-08,94,38,29,26


Here you can see that we have that our values are sorted in ascension by Walk first, and then Swim as well. 

___

## Other types of Join

Besides concat, you can also use `join()`, `merge()` and `append()` to combine different DataFrames. To go through these concepts, we will read in a new dataset on grades.

In [48]:
# Read in csv
grades = pd.read_csv('grades.csv')

In [49]:
# View the data
grades.head()

,assignment,weight,marks,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1,0.1,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0.1,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0.1,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0.1,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0.1,92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's begin by setting our index to be the assignment numbers.

In [50]:
grades.set_index('assignment', inplace=True)

### Data Cleaning

In [51]:
# Clean up the data 
grades = grades[['weight', 'marks']]

grades.head()

,weight,marks
assignment,,
1,0.1,93
2,0.1,44
3,0.1,75
4,0.1,78
5,0.1,92


Now, we want to determine the final grade of the student using these 10 assignments. First, we check to see if the weighting of these grades adds up to 100 to make sure we aren't missing any data.

### Summing over Rows/Columns

In [52]:
# Summing a column against all the rows (axis=0)
grades.weight.sum(axis=0)

0.8

This is less useful in our scenario here, but if you wanted to sum against all the columns, you would just set axis=1 instead of axis=0. 

In [53]:
grades.sum(axis=1)

assignment
1    93.1
2    44.1
3    75.1
4    78.1
5    92.1
6    90.1
7    95.1
8    57.1
dtype: float64

So we know that we are missing some data here. We happen to know that the remaining data is:
* Assignment 9: 78
* Assignment 10: 83. 

This data is stored in another dataframe called missing_data

In [54]:
missing_data = pd.DataFrame({'assignment': [9,10],
                             'weight': [0.1, 0.1],
                             'marks': [78, 93]})

missing_data.set_index('assignment', inplace=True)
missing_data.head()

,weight,marks
assignment,,
9,0.1,78
10,0.1,93


We use `df1.append(df2)` to add our missing data onto the end of our existing dataframe. This creates a new dataframe so we will call this `complete_grades`.

In [55]:
complete_grades = grades.append(missing_data)

complete_grades

,weight,marks
assignment,,
1,0.1,93
2,0.1,44
3,0.1,75
4,0.1,78
5,0.1,92
6,0.1,90
7,0.1,95
8,0.1,57
9,0.1,78


Since our index was already 9, 10 in the missing_data DataFrame, then we would not ignore the index. 

There is also `merge` and `join` but we won't go into detail on those two methods. If you are interested in the differences you can take a look [here](https://www.geeksforgeeks.org/what-is-the-difference-between-join-and-merge-in-pandas/#:~:text=Both%20join%20and%20merge%20can,join%20on%20for%20both%20dataframes.)

Next, let's calculate the weighting of each assignment from this student's grades. This would be multiplying the weight of each assignment by the marks and we'll store this in a new column, `mark_weight`.

___

## Multiplying Columns

In [56]:
# Multiply weight and marks 
complete_grades['mark_weight'] = complete_grades['weight'] * complete_grades['marks']

In [57]:
# View the data
complete_grades.head()

,weight,marks,mark_weight
assignment,,,
1,0.1,93,9.3
2,0.1,44,4.4
3,0.1,75,7.5
4,0.1,78,7.8
5,0.1,92,9.2


Adding, subtracting, multiplying and dividing columns does it element-by-element. 

Now, let's calculate the students final grade from these assignments.

In [58]:
# Calculate final grade
complete_grades.mark_weight.sum(axis=0)

79.5

Now, what if you wanted to see the percent change in your grades over the assignments. There is a quick and easy way to do that! 

___

## Percentage Changes

In [59]:
complete_grades.marks.pct_change()

assignment
1          NaN
2    -0.526882
3     0.704545
4     0.040000
5     0.179487
6    -0.021739
7     0.055556
8    -0.400000
9     0.368421
10    0.192308
Name: marks, dtype: float64

Note that it doesn't count the percentage change for the first assignment because there is no assignment 0 to reference to. We can see that for the most part the assignment grades are improving throughout the term. 

___

## Filtering

We went through some basic filtering with masks. There are a few other ways to filter your index or columns if they contain strings. For these purposes, we will use another set of data.

Suppose you make a diary of all the fruits you ate from Jan to March 2021. We want to do a quick analysis on these fruits to see what types of fruits you prefer.

In [60]:
# Import the data
fruits = pd.read_csv('fruits.csv')
fruits.set_index('Date', inplace=True)

fruits.head()

,Fruit
Date,
2021-01-01,apple
2021-01-02,banana
2021-01-03,strawberry
2021-01-04,pears
2021-01-05,pineapple


In [61]:
# What is the distribution of fruits
fruits.Fruit.value_counts()

strawberry    15
apple         11
pineapple     10
kiwi          10
orange        10
watermelon     9
banana         7
grapes         7
pears          6
peach          5
Name: Fruit, dtype: int64

Clearly, strawberry is your favorite fruit. What if I wanted to see what fruit I ate the most in February? 

In [62]:
feb_fruits = fruits.filter(like='2021-02', axis=0)

feb_fruits.head()

,Fruit
Date,
2021-02-01,kiwi
2021-02-02,watermelon
2021-02-03,strawberry
2021-02-04,peach
2021-02-05,kiwi


This essentially checks the index (axis=0) so all rows to see if '2021-02' is in the Dates. This is a quick way to get your data when your index is a string.

What if I wanted to get all the dates where I ate strawberries? 

In [63]:
fruits[fruits.Fruit == 'strawberry']

,Fruit
Date,
2021-01-03,strawberry
2021-01-14,strawberry
2021-01-22,strawberry
2021-01-29,strawberry
2021-02-03,strawberry
2021-02-07,strawberry
2021-02-12,strawberry
2021-02-16,strawberry
2021-02-18,strawberry


What if I wanted to check for strawberries and apples?

There are two methods you can use, you can do this with a OR in your mask or you can use `df.isin` which takes a list and outputs when the data has values in that list.

In [64]:
mask = fruits.Fruit.isin(['apple', 'strawberry'])

mask.head()

Date
2021-01-01     True
2021-01-02    False
2021-01-03     True
2021-01-04    False
2021-01-05    False
Name: Fruit, dtype: bool

A mask basically gives you a True or False for each row of your DataFrame depending on if the condition holds true or not.

In [65]:
# All the dates you ate apples or strawberries
fruits[mask]

,Fruit
Date,
2021-01-01,apple
2021-01-03,strawberry
2021-01-14,strawberry
2021-01-15,apple
2021-01-17,apple
2021-01-22,strawberry
2021-01-27,apple
2021-01-29,strawberry
2021-01-30,apple


___

# Exercises

### Yahoo Finance

Exercise 1: Graph a 2x1 subplot with open and close prices for UBER. Display the quarterly dates for the x-axis. Use Yahoo Finance to get your data from '2021-01-01' to '2022-01-01'.

### Activity Data

Exercise 2: Calculate the median time spent on each activity and add it to the `activity_stats` DataFrame as a new column.

Exercise 3: Calculate the average total exercise time that you spend during this month. Which date did you spend the most time exercising? What about the least time exercising? 

Exercise 4: What is your monthly average time spent exercising?

### Grades Data

Exercise 5: Calculate how much of their total grade the student lost in each assignment, i.e. the difference between the weight of the assignment and the weight from their mark.

Exercise 6: Plot their marks over time between the assignments.

### Fruits Data

Exercise 7: Plot the distribution of the fruits over time

Exercise 8: Get the total number of times you ate strawberries, apples, and pineapples per month. 

**HINT:** You can access the strings of a column by using `df.column_name.str` and then apply the slicing to get the month of each date.